In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.c

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Module_Sixteen_Challenge").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Sports_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

In [4]:
# Filter dataframe for total votes greater than or equal to 20.
over_twenty_df = df.filter(df.total_votes >=20)

In [5]:
# Filter dataframe for helpful_votes ratio greater than or equal to 50.
helpful_df = over_twenty_df.filter((over_twenty_df.helpful_votes / over_twenty_df.total_votes) >= 0.5)

In [6]:
# Create paid vine dataframe.
vine_df = helpful_df.filter(helpful_df.vine == 'Y')

In [7]:
# Create unpaid review dataframe.
unpaid_df = helpful_df.filter(helpful_df.vine == 'N')

In [8]:
# Total number of paid reviews.
paid_reviews = vine_df.count()
paid_reviews

334

In [9]:
# Total number of paid 5 star reviews.
paid_five_star = vine_df.filter(vine_df.star_rating == 5).count()
paid_five_star

139

In [10]:
# Percentage of paid 5 star reviews.
paid_percent = (paid_five_star / paid_reviews) * 100
paid_percent

41.61676646706587

In [12]:
# Total number of unpaid reviews.
unpaid_reviews = unpaid_df.count()
unpaid_reviews

61614

In [13]:
# Total number of unpaid 5 star reviews.
unpaid_five_star = unpaid_df.filter(unpaid_df.star_rating == 5).count()
unpaid_five_star

32665

In [14]:
# Percentage of unpaid 5 star reviews.
unpaid_percent = (unpaid_five_star / unpaid_reviews) * 100
unpaid_percent

53.01554841432142